## Imports

In [1]:
import csv
import re
from io import BytesIO
import requests
import pdfplumber
import os
import pikepdf
import pandas as pd
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader, PdfWriter
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import Choropleth
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from fuzzywuzzy import process
import geopandas as gpd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

In [2]:
p = Path.cwd()
data_folder = p.parent.parent / "data"
shape_folder = p.parent.parent / "shape"

# Leitura do ficheiro

In [3]:
file_path = data_folder / "EventosSemAltitudeETal.feather"
Inc = pd.read_feather(file_path)

Inc["concelho"] = Inc["concelho"].str.replace('-', ' ')
Inc

,codigo_do_relatorio,concelho,data_do_incidente,nivel_de_tensao,causa_do_incidente,duracao_incidente_min_,n_º_clientes_afetados,tiepi_mt_min_,saifi_mt_#_,saidi_mt_min_,...,sen_mes,cos_mes,t_med,t_max,t_min,dd_med,dd_ffx,ff_med,ff_max,pr_qtd
0,EDPD_2018_JAN_T_1,ALMADA,2018-01-02,MT,Escavações,55,828,0.002935,0.000044,0.002456,...,0.207912,0.978148,10.70,17.40,2.600,278.25,289.50,2.0,7.800,0.000000
1,EDPD_2018_JAN_T_2,SERTA,2018-01-02,MT,Abate De Árvores,163,549,0.002933,0.000177,0.016501,...,0.207912,0.978148,10.60,13.80,6.200,249.00,270.00,2.8,8.100,0.000000
2,EDPD_2018_JAN_T_3,FELGUEIRAS,2018-01-04,MT,Deslizamento De Terras,596,3459,0.019228,0.000266,0.035905,...,0.207912,0.978148,14.00,14.80,12.900,197.00,180.00,3.9,14.400,4.000000
3,EDPD_2018_JAN_T_4,TORRES VEDRAS,2018-01-05,BT,Veículos,178,102,0.000000,0.000000,0.000000,...,0.207912,0.978148,12.40,15.10,9.100,288.00,225.00,3.6,13.900,13.200000
4,EDPD_2018_JAN_T_5,NISA,2018-01-09,MT,Aves,119,1731,0.004422,0.000155,0.010038,...,0.207912,0.978148,4.90,7.80,2.200,137.00,118.00,4.1,15.400,15.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,EDPD_2020_DEZ_T_21,POMBAL,2020-12-24,MT,Abate De Árvores,139,2517,0.005941,0.000461,0.017245,...,0.587785,-0.809017,7.90,12.20,2.100,7.00,351.00,1.7,7.600,0.000000
2918,EDPD_2020_DEZ_T_22,VILA NOVA DE GAIA,2020-12-30,MT,Aves,81,3641,0.011914,0.000290,0.011131,...,0.587785,-0.809017,4.45,9.05,0.275,239.00,186.25,1.5,4.825,0.787500
2919,EDPD_2020_DEZ_T_23,ALBUFEIRA,2020-12-28,MT,Escavações,81,3175,0.005974,0.000086,0.005940,...,0.587785,-0.809017,13.20,15.30,7.700,285.00,293.00,5.8,14.100,0.100000
2920,EDPD_2020_DEZ_T_24,CASTRO MARIM,2020-12-28,MT,Veículos,239,591,0.006928,0.000054,0.011528,...,0.587785,-0.809017,13.20,15.30,7.700,285.00,293.00,5.8,14.100,0.100000


In [4]:
Inc[Inc["concelho"].str.contains("LOULE", case=False)]["concelho"].unique()

array(['LOULE'], dtype=object)

# Altura Máxima

In [5]:
file_path = data_folder / "AltitudeConcelhoMax.xlsx"
MaxAlt = pd.read_excel(file_path, header=11, usecols=range(1, 3), nrows=321)
MaxAlt.dropna(inplace=True)
MaxAlt.rename(columns = {"Anos": "concelho", 2023: "MaxHeight"}, inplace=True)
MaxAlt["concelho"] = MaxAlt["concelho"].str.upper()

def remover_pontuacao(text):
    text = text.replace('Ç', 'C').replace('Ã', 'A').replace('É', 'E').replace('Â', 'A').replace('Ô', 'O').replace('Á', 'A').replace('Ú', 'U').replace('Ê', 'E').replace('Ó', 'O').replace('Í', 'I')
    return text
    
MaxAlt["concelho"] = MaxAlt["concelho"].apply(remover_pontuacao)
MaxAlt["concelho"] = MaxAlt["concelho"].str.replace('-', ' ')

MaxAlt

,concelho,MaxHeight
0,ARCOS DE VALDEVEZ,1416.0
1,CAMINHA,805.0
2,MELGACO,1336.0
3,MONCAO,1114.0
4,PAREDES DE COURA,883.0
...,...,...
303,RIBEIRA BRAVA,1725.0
304,SANTA CRUZ,1415.0
305,SANTANA,1862.0
306,SAO VICENTE,1725.0


In [6]:
not_in_Inc = ~MaxAlt["concelho"].isin(Inc["concelho"])
values_not_in_Inc = MaxAlt[not_in_Inc]
values_not_in_Inc.head()

,concelho,MaxHeight
4,PAREDES DE COURA,883.0
19,MONDIM DE BASTO,1307.0
49,CASTELO DE PAIVA,694.0
59,ARMAMAR,955.0
61,FREIXO DE ESPADA À CINTA,885.0


In [7]:
MaxAlt[MaxAlt["concelho"].str.contains("VO", case=False)]["concelho"].unique()

array(['POVOA DE LANHOSO', 'POVOA DE VARZIM', 'TORRE DE MONCORVO',
       'ILHAVO', 'SEVER DO VOUGA', 'MIRANDA DO CORVO', 'VOUZELA', 'EVORA',
       'MONTEMOR O NOVO', 'POVOACAO', 'CORVO'], dtype=object)

In [8]:
SemiInc = pd.merge(Inc, MaxAlt, on=['concelho'], how='left')
SemiInc

,codigo_do_relatorio,concelho,data_do_incidente,nivel_de_tensao,causa_do_incidente,duracao_incidente_min_,n_º_clientes_afetados,tiepi_mt_min_,saifi_mt_#_,saidi_mt_min_,...,cos_mes,t_med,t_max,t_min,dd_med,dd_ffx,ff_med,ff_max,pr_qtd,MaxHeight
0,EDPD_2018_JAN_T_1,ALMADA,2018-01-02,MT,Escavações,55,828,0.002935,0.000044,0.002456,...,0.978148,10.70,17.40,2.600,278.25,289.50,2.0,7.800,0.000000,125.0
1,EDPD_2018_JAN_T_2,SERTA,2018-01-02,MT,Abate De Árvores,163,549,0.002933,0.000177,0.016501,...,0.978148,10.60,13.80,6.200,249.00,270.00,2.8,8.100,0.000000,1084.0
2,EDPD_2018_JAN_T_3,FELGUEIRAS,2018-01-04,MT,Deslizamento De Terras,596,3459,0.019228,0.000266,0.035905,...,0.978148,14.00,14.80,12.900,197.00,180.00,3.9,14.400,4.000000,575.0
3,EDPD_2018_JAN_T_4,TORRES VEDRAS,2018-01-05,BT,Veículos,178,102,0.000000,0.000000,0.000000,...,0.978148,12.40,15.10,9.100,288.00,225.00,3.6,13.900,13.200000,394.0
4,EDPD_2018_JAN_T_5,NISA,2018-01-09,MT,Aves,119,1731,0.004422,0.000155,0.010038,...,0.978148,4.90,7.80,2.200,137.00,118.00,4.1,15.400,15.900000,463.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,EDPD_2020_DEZ_T_21,POMBAL,2020-12-24,MT,Abate De Árvores,139,2517,0.005941,0.000461,0.017245,...,-0.809017,7.90,12.20,2.100,7.00,351.00,1.7,7.600,0.000000,560.0
2918,EDPD_2020_DEZ_T_22,VILA NOVA DE GAIA,2020-12-30,MT,Aves,81,3641,0.011914,0.000290,0.011131,...,-0.809017,4.45,9.05,0.275,239.00,186.25,1.5,4.825,0.787500,262.0
2919,EDPD_2020_DEZ_T_23,ALBUFEIRA,2020-12-28,MT,Escavações,81,3175,0.005974,0.000086,0.005940,...,-0.809017,13.20,15.30,7.700,285.00,293.00,5.8,14.100,0.100000,227.0
2920,EDPD_2020_DEZ_T_24,CASTRO MARIM,2020-12-28,MT,Veículos,239,591,0.006928,0.000054,0.011528,...,-0.809017,13.20,15.30,7.700,285.00,293.00,5.8,14.100,0.100000,276.0


In [9]:
SemiInc[SemiInc['MaxHeight'].isna()]

,codigo_do_relatorio,concelho,data_do_incidente,nivel_de_tensao,causa_do_incidente,duracao_incidente_min_,n_º_clientes_afetados,tiepi_mt_min_,saifi_mt_#_,saidi_mt_min_,...,cos_mes,t_med,t_max,t_min,dd_med,dd_ffx,ff_med,ff_max,pr_qtd,MaxHeight


# Altura Minima

In [10]:
file_path = data_folder / "AltitudeConcelhoMin.xlsx"
MinAlt = pd.read_excel(file_path, header=11, usecols=range(1, 3), nrows=321)
MinAlt.dropna(inplace=True)
MinAlt.rename(columns = {"Anos": "concelho", 2023: "MinHeight"}, inplace=True)
MinAlt["concelho"] = MinAlt["concelho"].str.upper()

def remover_pontuacao(text):
    text = text.replace('Ç', 'C').replace('Ã', 'A').replace('É', 'E').replace('Â', 'A').replace('Ô', 'O').replace('Á', 'A').replace('Ú', 'U').replace('Ê', 'E').replace('Ó', 'O').replace('Í', 'I')
    return text
    
MinAlt["concelho"] = MinAlt["concelho"].apply(remover_pontuacao)
MinAlt["concelho"] = MinAlt["concelho"].str.replace('-', ' ')

MinAlt

,concelho,MinHeight
0,ARCOS DE VALDEVEZ,17.0
1,CAMINHA,0.0
2,MELGACO,25.0
3,MONCAO,9.0
4,PAREDES DE COURA,125.0
...,...,...
303,RIBEIRA BRAVA,0.0
304,SANTA CRUZ,0.0
305,SANTANA,0.0
306,SAO VICENTE,0.0


In [11]:
not_in_Inc = ~MinAlt["concelho"].isin(SemiInc["concelho"])
values_not_in_Inc = MinAlt[not_in_Inc]
values_not_in_Inc.head()

,concelho,MinHeight
4,PAREDES DE COURA,125.0
19,MONDIM DE BASTO,100.0
49,CASTELO DE PAIVA,25.0
59,ARMAMAR,75.0
61,FREIXO DE ESPADA À CINTA,124.0


In [12]:
Completo = pd.merge(SemiInc, MinAlt, on=['concelho'], how='left')
Completo

,codigo_do_relatorio,concelho,data_do_incidente,nivel_de_tensao,causa_do_incidente,duracao_incidente_min_,n_º_clientes_afetados,tiepi_mt_min_,saifi_mt_#_,saidi_mt_min_,...,t_med,t_max,t_min,dd_med,dd_ffx,ff_med,ff_max,pr_qtd,MaxHeight,MinHeight
0,EDPD_2018_JAN_T_1,ALMADA,2018-01-02,MT,Escavações,55,828,0.002935,0.000044,0.002456,...,10.70,17.40,2.600,278.25,289.50,2.0,7.800,0.000000,125.0,0.0
1,EDPD_2018_JAN_T_2,SERTA,2018-01-02,MT,Abate De Árvores,163,549,0.002933,0.000177,0.016501,...,10.60,13.80,6.200,249.00,270.00,2.8,8.100,0.000000,1084.0,125.0
2,EDPD_2018_JAN_T_3,FELGUEIRAS,2018-01-04,MT,Deslizamento De Terras,596,3459,0.019228,0.000266,0.035905,...,14.00,14.80,12.900,197.00,180.00,3.9,14.400,4.000000,575.0,145.0
3,EDPD_2018_JAN_T_4,TORRES VEDRAS,2018-01-05,BT,Veículos,178,102,0.000000,0.000000,0.000000,...,12.40,15.10,9.100,288.00,225.00,3.6,13.900,13.200000,394.0,0.0
4,EDPD_2018_JAN_T_5,NISA,2018-01-09,MT,Aves,119,1731,0.004422,0.000155,0.010038,...,4.90,7.80,2.200,137.00,118.00,4.1,15.400,15.900000,463.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,EDPD_2020_DEZ_T_21,POMBAL,2020-12-24,MT,Abate De Árvores,139,2517,0.005941,0.000461,0.017245,...,7.90,12.20,2.100,7.00,351.00,1.7,7.600,0.000000,560.0,0.0
2918,EDPD_2020_DEZ_T_22,VILA NOVA DE GAIA,2020-12-30,MT,Aves,81,3641,0.011914,0.000290,0.011131,...,4.45,9.05,0.275,239.00,186.25,1.5,4.825,0.787500,262.0,0.0
2919,EDPD_2020_DEZ_T_23,ALBUFEIRA,2020-12-28,MT,Escavações,81,3175,0.005974,0.000086,0.005940,...,13.20,15.30,7.700,285.00,293.00,5.8,14.100,0.100000,227.0,0.0
2920,EDPD_2020_DEZ_T_24,CASTRO MARIM,2020-12-28,MT,Veículos,239,591,0.006928,0.000054,0.011528,...,13.20,15.30,7.700,285.00,293.00,5.8,14.100,0.100000,276.0,0.0


In [13]:
Completo[Completo['MinHeight'].isna()]

,codigo_do_relatorio,concelho,data_do_incidente,nivel_de_tensao,causa_do_incidente,duracao_incidente_min_,n_º_clientes_afetados,tiepi_mt_min_,saifi_mt_#_,saidi_mt_min_,...,t_med,t_max,t_min,dd_med,dd_ffx,ff_med,ff_max,pr_qtd,MaxHeight,MinHeight


In [14]:
Completo

,codigo_do_relatorio,concelho,data_do_incidente,nivel_de_tensao,causa_do_incidente,duracao_incidente_min_,n_º_clientes_afetados,tiepi_mt_min_,saifi_mt_#_,saidi_mt_min_,...,t_med,t_max,t_min,dd_med,dd_ffx,ff_med,ff_max,pr_qtd,MaxHeight,MinHeight
0,EDPD_2018_JAN_T_1,ALMADA,2018-01-02,MT,Escavações,55,828,0.002935,0.000044,0.002456,...,10.70,17.40,2.600,278.25,289.50,2.0,7.800,0.000000,125.0,0.0
1,EDPD_2018_JAN_T_2,SERTA,2018-01-02,MT,Abate De Árvores,163,549,0.002933,0.000177,0.016501,...,10.60,13.80,6.200,249.00,270.00,2.8,8.100,0.000000,1084.0,125.0
2,EDPD_2018_JAN_T_3,FELGUEIRAS,2018-01-04,MT,Deslizamento De Terras,596,3459,0.019228,0.000266,0.035905,...,14.00,14.80,12.900,197.00,180.00,3.9,14.400,4.000000,575.0,145.0
3,EDPD_2018_JAN_T_4,TORRES VEDRAS,2018-01-05,BT,Veículos,178,102,0.000000,0.000000,0.000000,...,12.40,15.10,9.100,288.00,225.00,3.6,13.900,13.200000,394.0,0.0
4,EDPD_2018_JAN_T_5,NISA,2018-01-09,MT,Aves,119,1731,0.004422,0.000155,0.010038,...,4.90,7.80,2.200,137.00,118.00,4.1,15.400,15.900000,463.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,EDPD_2020_DEZ_T_21,POMBAL,2020-12-24,MT,Abate De Árvores,139,2517,0.005941,0.000461,0.017245,...,7.90,12.20,2.100,7.00,351.00,1.7,7.600,0.000000,560.0,0.0
2918,EDPD_2020_DEZ_T_22,VILA NOVA DE GAIA,2020-12-30,MT,Aves,81,3641,0.011914,0.000290,0.011131,...,4.45,9.05,0.275,239.00,186.25,1.5,4.825,0.787500,262.0,0.0
2919,EDPD_2020_DEZ_T_23,ALBUFEIRA,2020-12-28,MT,Escavações,81,3175,0.005974,0.000086,0.005940,...,13.20,15.30,7.700,285.00,293.00,5.8,14.100,0.100000,227.0,0.0
2920,EDPD_2020_DEZ_T_24,CASTRO MARIM,2020-12-28,MT,Veículos,239,591,0.006928,0.000054,0.011528,...,13.20,15.30,7.700,285.00,293.00,5.8,14.100,0.100000,276.0,0.0


In [15]:
# Substituir os valores específicos na coluna
Completo.loc[Completo["duracao_incidente_min_"] == "UÇÕES249", "duracao_incidente_min_"] = 249
Completo.loc[Completo["duracao_incidente_min_"] == "UÇÕES182", "duracao_incidente_min_"] = 182
Completo.loc[Completo["duracao_incidente_min_"] == "UÇÕES288", "duracao_incidente_min_"] = 288
Completo["duracao_incidente_min_"] = Completo["duracao_incidente_min_"].astype(int)

In [16]:
Completo.to_feather("EventosFinalCompleto.feather")